<a href="https://colab.research.google.com/github/Yonaki97/DataScience/blob/main/SpotifyAnalisis_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import pandas as pd
df = pd.read_csv('spotify_churn_dataset.csv')

link dataset : https://www.kaggle.com/datasets/nabihazahid/spotify-dataset-for-churn-analysis

#PRE PROCESSING DATA

In [ ]:
df = pd.read_csv('spotify_churn_dataset.csv')

In [ ]:
df.shape

(8000, 12)

In [ ]:
df.head()

,user_id,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,1,Female,54,CA,Free,26,23,0.20,Desktop,31,0,1
1,2,Other,33,DE,Family,141,62,0.34,Web,0,1,0
2,3,Male,38,AU,Premium,199,38,0.04,Mobile,0,1,1
3,4,Female,22,CA,Student,36,2,0.31,Mobile,0,1,0
4,5,Other,29,US,Family,250,57,0.36,Mobile,0,1,1


In [ ]:
df.drop("user_id", inplace=True, axis=1)

In [ ]:
# mengganti spasi dengan under score
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

In [ ]:
for col in df.columns:
  if df[col].dtype == object:
      df[col] = df[col].str.lower()

In [ ]:
df.head()

,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,female,54,ca,free,26,23,0.20,desktop,31,0,1
1,other,33,de,family,141,62,0.34,web,0,1,0
2,male,38,au,premium,199,38,0.04,mobile,0,1,1
3,female,22,ca,student,36,2,0.31,mobile,0,1,0
4,other,29,us,family,250,57,0.36,mobile,0,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 8000 non-null   object 
 1   age                    8000 non-null   int64  
 2   country                8000 non-null   object 
 3   subscription_type      8000 non-null   object 
 4   listening_time         8000 non-null   int64  
 5   songs_played_per_day   8000 non-null   int64  
 6   skip_rate              8000 non-null   float64
 7   device_type            8000 non-null   object 
 8   ads_listened_per_week  8000 non-null   int64  
 9   offline_listening      8000 non-null   int64  
 10  is_churned             8000 non-null   int64  
dtypes: float64(1), int64(6), object(4)
memory usage: 687.6+ KB


In [ ]:
df.isnull().sum()

,0
gender,0
age,0
country,0
subscription_type,0
listening_time,0
songs_played_per_day,0
skip_rate,0
device_type,0
ads_listened_per_week,0
offline_listening,0


In [ ]:
duplicated = df.drop_duplicates()

In [ ]:
print("Sebelum:", len(duplicated))
df.drop_duplicates(inplace=True)
print("Sesudah:", len(duplicated))

Sebelum: 8000
Sesudah: 8000


In [ ]:
df.describe()

,age,listening_time,songs_played_per_day,skip_rate,ads_listened_per_week,offline_listening,is_churned
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,37.662125,154.068250,50.127250,0.300127,6.943875,0.747750,0.258875
std,12.740359,84.015596,28.449762,0.173594,13.617953,0.434331,0.438044
min,16.000000,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,26.000000,81.000000,25.000000,0.150000,0.000000,0.000000,0.000000
50%,38.000000,154.000000,50.000000,0.300000,0.000000,1.000000,0.000000
75%,49.000000,227.000000,75.000000,0.450000,5.000000,1.000000,1.000000
max,59.000000,299.000000,99.000000,0.600000,49.000000,1.000000,1.000000


In [ ]:
num_features = ["age","listening_time",
                "songs_played_per_day",
                "skip_rate","ads_listened_per_week",
                "offline_listening"]

cat_features = ["gender",
                "country",
                "subscription_type",
                "device_type"]

In [ ]:
def checking_outlier(list_feature, df, dataset_name):
    outlier_info = []

    for feature in list_feature:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)][feature]

        if len(outliers) > 0:
            outlier_info.append({
                "Feature": feature,
                "Outlier Count": len(outliers),
                # "Outlier Detail": outliers.tolist()
            })

    if len(outlier_info) == 0:
        print("✅ No outliers detected in the selected features.")
    else:
        return pd.DataFrame(outlier_info)

checking_outlier(list_feature=num_features, df=df, dataset_name="Data")

,Feature,Outlier Count
0,ads_listened_per_week,1683


In [ ]:
def remove_outliers_iqr(df, list_feature):
    df_clean = df.copy()
    for feature in list_feature:
        Q1 = df_clean[feature].quantile(0.25)
        Q3 = df_clean[feature].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Filter hanya nilai dalam batas
        df_clean = df_clean[(df_clean[feature] >= lower_bound) & (df_clean[feature] <= upper_bound)]

    print(f"✅ Outliers removed. Remaining rows: {len(df_clean)} / {len(df)}")
    return df_clean

In [ ]:
Scaler = MinMaxScaler()
df[num_features]= Scaler.fit_transform(df[num_features])

In [ ]:
df.head()

,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,female,0.883721,ca,free,0.055363,0.224490,0.333333,desktop,0.632653,0.0,1
1,other,0.395349,de,family,0.453287,0.622449,0.566667,web,0.000000,1.0,0
2,male,0.511628,au,premium,0.653979,0.377551,0.066667,mobile,0.000000,1.0,1
3,female,0.139535,ca,student,0.089965,0.010204,0.516667,mobile,0.000000,1.0,0
4,other,0.302326,us,family,0.830450,0.571429,0.600000,mobile,0.000000,1.0,1


In [ ]:
# Scaler = StandardScaler()
# df[num_features]= Scaler.fit_transform(df[num_features])

In [ ]:
df = pd.get_dummies(df, columns=cat_features, drop_first=True)

#####HASIL SEMENTARA

In [ ]:
df.head()

,age,listening_time,songs_played_per_day,skip_rate,ads_listened_per_week,offline_listening,is_churned,gender_male,gender_other,country_ca,...,country_fr,country_in,country_pk,country_uk,country_us,subscription_type_free,subscription_type_premium,subscription_type_student,device_type_mobile,device_type_web
0,0.883721,0.055363,0.224490,0.333333,0.632653,0.0,1,False,False,True,...,False,False,False,False,False,True,False,False,False,False
1,0.395349,0.453287,0.622449,0.566667,0.000000,1.0,0,False,True,False,...,False,False,False,False,False,False,False,False,False,True
2,0.511628,0.653979,0.377551,0.066667,0.000000,1.0,1,True,False,False,...,False,False,False,False,False,False,True,False,True,False
3,0.139535,0.089965,0.010204,0.516667,0.000000,1.0,0,False,False,True,...,False,False,False,False,False,False,False,True,True,False
4,0.302326,0.830450,0.571429,0.600000,0.000000,1.0,1,False,True,False,...,False,False,False,False,True,False,False,False,True,False


In [ ]:
df.tail()

,age,listening_time,songs_played_per_day,skip_rate,ads_listened_per_week,offline_listening,is_churned,gender_male,gender_other,country_ca,...,country_fr,country_in,country_pk,country_uk,country_us,subscription_type_free,subscription_type_premium,subscription_type_student,device_type_mobile,device_type_web
7995,0.651163,0.785467,0.357143,0.500000,0.000000,1.0,1,False,True,False,...,False,False,False,False,False,False,False,True,True,False
7996,0.418605,0.176471,0.642857,0.983333,0.000000,1.0,0,True,False,False,...,False,False,False,False,False,False,True,False,True,False
7997,0.023256,0.245675,0.622449,0.550000,0.102041,0.0,0,False,False,False,...,False,False,False,False,True,True,False,False,False,False
7998,0.418605,0.813149,0.948980,0.450000,0.000000,1.0,0,False,False,False,...,False,True,False,False,False,False,False,True,False,False
7999,0.674419,0.692042,0.683673,0.766667,0.122449,0.0,0,False,True,False,...,False,False,False,False,False,True,False,False,False,False


#Machine Learning Session

In [ ]:
df.iloc[4511]

,4511
age,0.813953
listening_time,0.301038
songs_played_per_day,0.357143
skip_rate,0.416667
ads_listened_per_week,0.204082
offline_listening,0.0
is_churned,0
gender_male,False
gender_other,True
country_ca,False


In [ ]:
X = df.drop('is_churned', axis=1)
y = df['is_churned']
X_Train, X_test, Y_Train, Y_test = train_test_split(X,y, test_size=0.2,random_state=42)
Y_train = np.reshape(Y_Train, (-1, 1))
Y_test = np.reshape(Y_test, (-1, 1))
X_Train.shape, Y_train.shape

((6400, 20), (6400, 1))

In [ ]:
model = LogisticRegression(max_iter=1000, solver='lbfgs', class_weight='balanced')
model.fit(X_Train, Y_Train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=42)
# rf.fit(X_Train, Y_Train)
# print(rf.score(X_test, Y_test))

# Kalau misal menggunakan Random Forest

In [ ]:
# ambil 5 data random dari test
sample_idx = X_test.sample(5, random_state=42)

# predict probability & predict class
sample_pred = model.predict(sample_idx)
sample_prob = model.predict_proba(sample_idx)

# tampilkan hasilnya bareng datanya
display(sample_idx)
print("Prediksi kelas:", sample_pred)
print("Probabilitas prediksi:", sample_prob)

,age,listening_time,songs_played_per_day,skip_rate,ads_listened_per_week,offline_listening,gender_male,gender_other,country_ca,country_de,country_fr,country_in,country_pk,country_uk,country_us,subscription_type_free,subscription_type_premium,subscription_type_student,device_type_mobile,device_type_web
4511,0.813953,0.301038,0.357143,0.416667,0.204082,0.0,False,True,False,False,False,False,False,True,False,True,False,False,False,False
5528,0.697674,0.048443,0.000000,0.566667,0.000000,1.0,False,True,False,False,False,False,False,False,True,False,True,False,False,False
7700,0.069767,0.217993,0.173469,0.900000,0.000000,1.0,False,False,False,False,False,False,False,True,False,False,False,False,True,False
5337,0.744186,0.584775,0.357143,0.566667,0.653061,0.0,False,False,False,True,False,False,False,False,False,True,False,False,False,True
6187,0.418605,0.356401,0.928571,0.566667,0.000000,1.0,False,True,False,True,False,False,False,False,False,False,True,False,True,False


Prediksi kelas: [0 0 1 0 1]
Probabilitas prediksi: [[0.54087885 0.45912115]
 [0.5178265  0.4821735 ]
 [0.47964931 0.52035069]
 [0.50122973 0.49877027]
 [0.47207256 0.52792744]]


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print("\n Accuracy Score:\n", accuracy_score(Y_test, y_pred))


 Accuracy Score:
 0.52125


In [ ]:
print("\nConfusion Matrix:\n", confusion_matrix(Y_test, y_pred))


Confusion Matrix:
 [[619 581]
 [185 215]]


In [ ]:
print("\nClassification Report:\n", classification_report(Y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.52      0.62      1200
           1       0.27      0.54      0.36       400

    accuracy                           0.52      1600
   macro avg       0.52      0.53      0.49      1600
weighted avg       0.64      0.52      0.55      1600



In [ ]:
pred = model.predict(X_test)

final = pd.DataFrame()
final['user_id'] = X_test.index
final['is_churned'] = pred
final.to_csv('is_churned.csv', index=False)